# Sigils of The Codex

By the end of this Code Along session, you will learn to run inferences through `Google's Cloud Vision API`, retrieve annotation labels, and sort them by recyclable categories. You will implement a **scoring system** to determine the best prediction for each image, and build a simple tool to **identify recyclable items** within images.

## Install Dependancies
We'll first need to install some dependancies. Here is a breakdown of the necessary modules:
- `google-cloud-vision`: The official Google Cloud Vision API client library for Python.

- `pillow`: A Python Imaging Library with Image Processing capabilities.

In [ ]:
%pip install google-cloud-vision pillow

## Dependancy Import and Environment Setup
We'll authenticate our Google Cloud Vision API client using the `serviceAccountKey.json` file.

In [ ]:
import os, json, tempfile
from google.cloud import vision
from PIL import Image

# Set Credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'serviceAccountKey.json'

## Load Data
`config.json` → maps categories (e.g. Plastic & Paper) to Google's Cloud Vision API labels.

In [ ]:
with open('config.json', 'r') as f:
    config = json.load(f)

## Pre-processing (Image Optimization)
Resize and convert all images to `JPG` format.

In [ ]:
def optimize_image(image_path, max_size=(800, 800)):
    try:
        with Image.open(image_path) as img:
            img.thumbnail(max_size)

            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".jpg")
            img.save(temp_file.name, "JPEG", quality=85)

            return temp_file.name
    except Exception as e:
        print(f"Error optimizing image: {e}")
        return image_path

## Object Localization
`ImageAnnotatorClient` performs **Object Localization** on the images. We'll construct a collection of annotation labels.

In [ ]:
def get_detected_objects(image_path):
    client = vision.ImageAnnotatorClient()
    try:
        with open(image_path, 'rb') as image_file:
            content = image_file.read()
        image = vision.Image(content=content)

        response = client.object_localization(image=image)
        objects = response.localized_object_annotations

        if response.error.message:
            raise Exception(f'Error: {response.error.message}')

        detected_objects = [obj.name for obj in objects]
        return detected_objects
    except Exception as e:
        return {'error': str(e)}

## Object Mapping
We'll map the detected objects to their corresponding categories to find the most relevant category for each image.

In [ ]:
def get_recyclable_categories(detected_objects):
    matching_categories = set()
    for category, terms in config.items():
        if any(obj in terms for obj in detected_objects):
            matching_categories.add(category)
    return list(matching_categories) if matching_categories else None

## Handling Edge-Case (Multiple Objects Detected)
In the likely scenario that multiple objects are detected in an image, we will implement a **scoring system** to determine the best-fit category based on the **object overlaps**. In the scenario that there is still a tie between multiple categories, we will perform `Label Detection` on the image to get a more granular analysis of the detected objects. This will help us identify the most relevant category based on the most occurences of label matches across a wider range of generic labels.

`ImageAnnotatorClient` performs **Label Detection** on the images. We'll construct a collection of annotation labels.

In [ ]:
def granular_analysis_to_resolve_tie(file_name, image_path, tied_categories):
    client = vision.ImageAnnotatorClient()
    try:
        with open(image_path, 'rb') as image_file:
            content = image_file.read()
        image = vision.Image(content=content)

        response = client.label_detection(image=image)
        labels = [label.description for label in response.label_annotations]

        if response.error.message:
            raise Exception(f'Error: {response.error.message}')

        label_match_count = {category: 0 for category in tied_categories}
        for category in tied_categories:
            terms = config.get(category, [])
            label_match_count[category] = sum(1 for label in labels if label in terms)

        best_category = max(label_match_count, key=label_match_count.get)
        return best_category if label_match_count[best_category] > 0 else None
    except Exception as e:
        return {'error': str(e)}

def get_best_fitting_category(file_name, image_path, detected_objects, matched_categories):
    category_match_count = {category: 0 for category in matched_categories}
    for category in matched_categories:
        terms = config.get(category, [])
        category_match_count[category] = sum(1 for obj in detected_objects if obj in terms)

    best_category = max(category_match_count, key=category_match_count.get)
    highest_count = category_match_count[best_category]

    tied_categories = [cat for cat, count in category_match_count.items() if count == highest_count]

    if len(tied_categories) > 1:
        return granular_analysis_to_resolve_tie(file_name, image_path, tied_categories)

    return best_category

## Putting it to the test!

Upload an image of a **paper bag** and run an inference through our pipeline.

In [ ]:
def analyze_image(image_path):
    optimized_path = optimize_image(image_path)

    detected_objects = get_detected_objects(optimized_path)
    if isinstance(detected_objects, dict) and 'error' in detected_objects:
        print(f"Error detecting objects: {detected_objects['error']}")
        return

    if not detected_objects:
        print("Category: No match")
        return

    recyclable_categories = get_recyclable_categories(detected_objects)
    if recyclable_categories:
        best_category = get_best_fitting_category(
            file_name=os.path.basename(image_path),
            image_path=image_path,
            detected_objects=detected_objects,
            matched_categories=recyclable_categories
        )
        if best_category:
            print(f"{best_category} is recyclable!")
            os.remove(optimized_path)
            return

    print("Category: No match")
    os.remove(optimized_path)

In [ ]:
image = "images/giftBag.jpg"
analyze_image(image)

## Understanding Annotation Labels
Try uploading an image of a **cardboard box** and run an inference through the pipeline. Do we still get a valid prediction?

In [ ]:
image = "images/cardboardBox.jpg"
analyze_image(image)

Our pipeline does not recognise the **cardboard box**, as we do not have a collection of recognised annotation labels for `Cardboard`. The Google Cloud Vision API has a large global collection of annotation labels, and our pipeline could not find any of them in our `config.json` file.

Let's fix that!

First, let's build a **helper function** that fetches labels for us.

In [ ]:
def fetch_labels(image_path):
    client = vision.ImageAnnotatorClient()

    try:
        with open(image_path, "rb") as image_file:
            content = image_file.read()
        image = vision.Image(content=content)

        response = client.label_detection(image=image)
        labels = [label.description for label in response.label_annotations]

        if response.error.message:
            raise Exception(f'Error: {response.error.message}')

        return labels

    except Exception as e:
        return {'error': str(e)}

Now, let's build a **helper function** that automates the population of our `config.json` file with the labels returned by Google's Cloud Vision API.

In [ ]:
def populate_config(category_type, image_file):
    client = vision.ImageAnnotatorClient()

    try:
        with open(image_file, "rb") as image_file_obj:
            content = image_file_obj.read()
        image = vision.Image(content=content)

        response = client.label_detection(image=image)
        labels = [label.description for label in response.label_annotations]

        if response.error.message:
            raise Exception(f'Error: {response.error.message}')

        if os.path.exists("config.json"):
            with open("config.json", "r") as f:
                config = json.load(f)
        else:
            config = {}

        if category_type not in config:
            config[category_type] = []

        existing_labels = set(config[category_type])
        new_labels = [label for label in labels if label not in existing_labels]

        if new_labels:
            config[category_type].extend(new_labels)

            with open("config.json", "w") as f:
                json.dump(config, f, indent=4)

        return {
            "category": category_type,
            "new_labels_added": len(new_labels),
        }

    except Exception as e:
        return {"error": str(e)}

Now, let's run the image of the **cardboard box** through our `populate_config` helper function.

In [ ]:
image = "images/cardboardBox.jpg"
category_type = "Cardboard"

print(populate_config(category_type, image))

Great! Now, the **cardboard box** is recognised as a recyclable item, and we have added the new labels to our `config.json` file. Let's run more new images through our pipeline to expand our array of recognised labels.

Why? This **improves reliability and accuracy** of our pipeline, ensuring it can handle a wider variety of recyclable items.

In [ ]:
images = ["images/cardboardBoxTrain_1.jpg", "images/cardboardBoxTrain_2.jpg", "images/cardboardBoxTrain_3.jpg"]

for image in images:
    new_labels = print(populate_config(category_type, image))

Now, let's test our improved pipeline with a **new** image of a **cardboard box**.

In [ ]:
image = "images/newCardboardBox.jpg"
analyze_image(image)

Great! Now we have a collection of recognised labels for `Cardboard` from Google's Cloud Vision API in our `config.json` file. Remember, the more annotation labels we have in our collection, the more **reliable** our system will be at identifying recyclable items. This could require hundreds of inferences!

Let's take a look at the annotation labels we have retrieved.

In [ ]:
print(config["Cardboard"])